# Google Colab

You can use the button below to open this notebook in Google Colab. Note that changes made to the notebook in Colab will not be reflected in Github, nor can the notebook be saved on Colab without first making a copy. 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nikitalokhmachev-ai/radio-map-estimation-public/blob/main/notebooks/Visualize_Inputs_Outputs.ipynb)

If opened in Colab, set `using_colab` to `True` in the code block below, then run the second and (optionally) third blocks. The second block will install the needed version of joblib for the data scaler to be loaded, then clone the github repository into Colab's local storage in order to load the models and other functions. The third block will connect to Google Drive (user login required), which allows the Colab notebook to read and write data to the drive (e.g. training data or evaluation results).

In [ ]:
using_colab = False

In [ ]:
if using_colab:
    %cd /content/
    !rm -rf /content/radio-map-estimation-public
    !git clone https://github.com/nikitalokhmachev-ai/radio-map-estimation-public.git
    !pip install -q -r /content/radio-map-estimation-public/requirements.txt

In [ ]:
if using_colab:
    from google.colab import drive
    drive.mount('/content/drive')

# Untar Validation Data

We visualize the inputs and outputs of the validation data, but you can use any data you choose.

In [ ]:
!tar -xkf '/Path/to/saved/tar/file' -C '/path/to/save/untarred/files'

# Import Packages

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

import os
import glob
import joblib
import random

In [ ]:
# Import model architectures and data structures

os.chdir('path/to/repository')
from data_utils import MapDataset

# Set Hyperparameters

In [ ]:
# Set random seed, define device

seed = 3
torch.manual_seed(seed)
torch.use_deterministic_algorithms(True)
np.random.seed(seed)
random.seed(seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Specify folder containing trained models
model_folder = '/folder/with/trained/models'

# Specify path to untarred validation data
val_data_folder = '/path/to/untarred/validation/data'

# Specify path to data scaler and load
scaler_path = 'scalers/minmax_scaler_zero_min134.joblib'
with open(scaler_path, 'rb') as f:
  scaler = joblib.load(f)

# Set folder to save visualizations
viz_folder = '/Path/to/save/visualizations'
if not os.path.exists(viz_folder):
    os.makedirs(viz_folder)

# Visualize Input and Ground Truth

The Sample Map and Environment Mask are used as inputs to the model. The complete Radio Map is the ground truth that the model seeks to recreate. We use the example map shown in the paper below, but this can be replaced with any image from the validation set or other dataset.

In [ ]:
# Example batch
example_batch = os.path.join(val_data_folder, 'test_0.01%_batch_0.pickle')
# Index of map within batch
i=37

# Load batch
t_x_points, t_channel_pows, t_y_masks = np.load(example_batch, allow_pickle=True)
# Select map within in batch
t_x_points = t_x_points[i:i+1]
t_channel_pows = t_channel_pows[i:i+1]
t_y_masks = t_y_masks[i:i+1]

# Manually preprocess map (this would normally be done by the MapDataset class)
t_y_points = t_channel_pows * t_y_masks
t_x_masks = t_x_points[:,1,:,:] == 1
t_x_points[:,0,:,:] = scaler.transform(t_x_points[:,0,:,:]) * t_x_masks
t_channel_pows = scaler.transform(t_channel_pows)
t_y_points = scaler.transform(t_y_points)

sample_map = t_x_points[0,0,:,:]
env_mask = t_x_points[0,1,:,:]
target = t_y_points[0,0,:,:]
target[env_mask==-1] = 1

# Visualize
fig, axs = plt.subplots(1,3, figsize=(6,5))
axs[0].imshow(sample_map, cmap='hot', vmin=0, vmax=1)
axs[0].set_title('Sampled Map')
axs[1].imshow(env_mask, cmap='binary')
axs[1].set_title('Environment Mask')
axs[2].imshow(target, cmap='hot', vmin=0, vmax=1)
axs[2].set_title('Complete Radio Map')
[ax.set_xticks([]) for ax in axs]
[ax.set_yticks([]) for ax in axs]
fig.tight_layout()
fig.show()

# Visualize Output and Intermediate Layers

In [ ]:
def get_model_output(x, channel_id, model, model_layer):
  #x: bs, c, h, w
  x = x.to(device)
  activation = {}
  def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

  model_layer.register_forward_hook(get_activation('out'))
  output = model(x)

  return activation['out'][0].permute(1,2,0).detach().cpu()[:,:,channel_id].unsqueeze(-1).numpy()

def visualize_layer(x, model, model_layer, nrows, ncols, figsize=(15, 15), out_folder=None, filename=None):
  n_channels = model_layer.out_channels
  fig, axs = plt.subplots(nrows, ncols, figsize=figsize)
  for i in range(nrows):
    for j in range(ncols):
      channel_id = i * ncols + j
      if channel_id < n_channels:
        axs[i, j].imshow(get_model_output(x, channel_id, model, model_layer))
        axs[i, j].set_title(str(channel_id))
        axs[i, j].axis('off')
      else:
        axs[i, j].axis('off')
  plt.tight_layout()
  if out_folder and filename:
    plt.savefig(os.path.join(out_folder, filename))
  plt.show()

def visualize_output(x, model, figsize=(5, 5), out_folder=None, filename=None):
  x_mask = x[:,1,:,:]
  plt.figure(figsize=figsize)
  plt.axis('off')
  plt.title('Model Output')
  prediction = model(x).reshape(1,32,32)
  prediction[x_mask==-1] = 1
  prediction = prediction.detach().cpu().numpy().transpose(1,2,0)
  plt.imshow(prediction, cmap='hot', vmin=0, vmax=1)
  plt.tight_layout()
  if out_folder and filename:
    plt.savefig(os.path.join(out_folder, filename))
  plt.show()

The code below visualizes either the output of the model (i.e. the predicted map) or the model representation at an intermediate layer. The user first specifies the model. If visualizing an intermediate layer, the user also specifies the layer from either the encoder or decoder. Layer names and attributes are printed out in the list below. Note that only Conv2d or ConvTranspose2d layers can be visualized.

In [ ]:
model_name = 'Baseline'
model = torch.load(os.path.join(model_folder, f'{model_name}.pth'), map_location=device)
model.eval()

encoder = model.encoder
decoder = model.decoder
print(encoder)
print()
print(decoder)

In [ ]:
# Convert input numpy array to tensor
x = torch.from_numpy(t_x_points).to(torch.float32).to(device)

In [ ]:
# Select layer and visualize channel outputs
model_layer = encoder.conv2d_1
visualize_layer(x, model, model_layer, nrows=5, ncols=6)

In [ ]:
# Visualize model output
visualize_output(x, model)